In [76]:
GROUP_ID = "04"
ALG = "ValItr"
TRACK_NAME = "../tracks/2-track.txt"
CRASH_POS = "NRST"

In [77]:
class RaceTrack:
    def __init__(self, file):
        with(open(file , "r") as f):
            lines = f.readlines()
            dims = lines[0]
            self.size = [0,0]
            self.size[0] = int(dims.split(",")[1].strip("\n"))
            self.size[1] = int(dims.split(",")[0])
            self.array = [[_ for y in range(self.size[1])] for x in range(self.size[0])]
            yIndex = 0
            for line in lines[1:]:
                xIndex = 0
                for char in line[:-1]:
                    self.array[xIndex][yIndex] = char
                    xIndex += 1
                yIndex += 1

    #prints track to command line, not for actual output
    def printTrack(self):
        for y in range(self.size[1]):
            line = ''
            for x in range(self.size[0]):
                line += self.array[x][y]
            print(line)


In [78]:
r = RaceTrack(TRACK_NAME)
r.printTrack()

##############################
####.................#########
###...................########
##....................########
#......................#######
#.......###########.....######
#......#############.....#####
#.....##############.....#####
#SSSSS#############.....######
#################......#######
###############......#########
##############.....###########
#############.....############
############.....#############
###########.....##############
##########.....###############
#########....#################
########....##################
######....####################
#####....###############FFFFF#
#####....###############.....#
#####.....#############......#
######.....###########.......#
#######......................#
########....................##
########...................###
#########.................####
##############################
